In [32]:
import cv2 
import numpy as np
import os 
import pandas as pd 
import torch 

from importlib import import_module 
from skp.toolbox.functions import load_model_from_config
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [49]:
class ICHDataset(Dataset):

    def __init__(self, df, data_dir):
        df_list = [_df for _, _df in df.groupby("SeriesInstanceUID")]
        self.series = df_list
        self.data_dir = data_dir 

    def __len__(self):
        return len(self.series)
    
    def __getitem__(self, idx):
        series = self.series[idx]
        series = series.sort_values("filepath", ascending=True)
        files = series.filepath.values
        array = np.stack([cv2.imread(os.path.join(self.data_dir, f), 0) for f in files], axis=0)
        array = np.concatenate([array[0][np.newaxis]] + [array] + [array[-1][np.newaxis]], axis=0)
        array = np.stack([array[:-2], array[1:-1], array[2:]], axis=1)
        array = torch.from_numpy(array).float()
        labels = torch.from_numpy(series[["any", "epidural", "intraparenchymal", "intraventricular", "subarachnoid", "subdural"]].values).long()
        return array, labels, series.SeriesInstanceUID.iloc[0]

In [3]:
cfg_name = "ich.cfg_slice_cls_pseudoseg_2dc_hard_labels"
cfg = import_module(f"skp.configs.{cfg_name}").cfg
cfg.pretrained = False 
cfg.load_pretrained_encoder = False 
cfg.enable_gradient_checkpointing = False 

weights_path = os.path.join(cfg.save_dir, cfg_name, "482cc224/fold0/checkpoints/last.ckpt")
model = load_model_from_config(cfg, weights_path, device="cuda:0", eval_mode=True)

Loading weights from /home/ian/projects/SKP/experiments/ich/ich.cfg_slice_cls_pseudoseg_2dc_hard_labels/482cc224/fold0/checkpoints/last.ckpt ...


In [7]:
df = pd.read_csv(cfg.annotations_file)
df.head()

,SOPInstanceUID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,filepath,PatientID,StudyInstanceUID,...,inner2,inner3,inner4,fold,ivh_mask,sdh_mask,any_mask,iph_mask,sah_mask,edh_mask
0,ID_36a144b7a,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0000...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
1,ID_9a319f990,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0001...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
2,ID_d8a328562,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0002...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
3,ID_f870dcfd0,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0003...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
4,ID_8db919b5c,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0004...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty


In [51]:
dataset = ICHDataset(df, cfg.data_dir)
loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=8, drop_last=False)

In [52]:
save_dir = "/mnt/stor/datasets/kaggle/rsna-intracranial-hemorrhage-detection/features_2dc_cls_pseudoseg/fold0/"
os.makedirs(save_dir, exist_ok=True)

In [58]:
for batch in tqdm(loader):
    array, labels, sid = batch
    array = array.to("cuda:0").squeeze(0)
    with torch.inference_mode():
        # extract feature maps, then apply pooling to final feature map
        features = model.classifier[0](model({"seg": {"x": array}}, return_features=True)["seg"]["features"][-1])
        features = features.cpu().numpy()
    np.save(os.path.join(save_dir, f"{sid[0]}_features.npy"), features)
    np.save(os.path.join(save_dir, f"{sid[0]}_labels.npy"), labels.numpy()[0])

100%|██████████| 20291/20291 [1:23:06<00:00,  4.07it/s]


In [54]:
sid

('ID_0000298a7d',)

In [42]:
features["seg"]["features"][-1].shape

torch.Size([40, 512, 16, 16])

In [14]:
df_list[0].sort_values("filepath", ascending=True)

,SOPInstanceUID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,filepath,PatientID,StudyInstanceUID,...,inner2,inner3,inner4,fold,ivh_mask,sdh_mask,any_mask,iph_mask,sah_mask,edh_mask
0,ID_36a144b7a,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0000...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
1,ID_9a319f990,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0001...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
2,ID_d8a328562,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0002...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
3,ID_f870dcfd0,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0003...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
4,ID_8db919b5c,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0004...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
5,ID_bb7aea41b,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0005...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
6,ID_371de8f49,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0006...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
7,ID_6b55e3e00,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0007...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
8,ID_da9a9b844,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0008...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
9,ID_cd15da2e3,0,0,0,0,0,0,ID_b9797064/ID_00b9e1961f/ID_0000298a7d/IM0009...,ID_b9797064,ID_00b9e1961f,...,0,0,4,1,empty,empty,empty,empty,empty,empty
